In [1]:
import sys
sys.argv = ['main.py']
from main import main
main()


[2025-10-14 12:02:41] 
[2025-10-14 12:02:41] ================================================================================
[2025-10-14 12:02:41] BASKETBALL SCRAPER STARTED
[2025-10-14 12:02:41] Time: 2025-10-14 12:02:41
[2025-10-14 12:02:41] Mode: QUICK
[2025-10-14 12:02:41] ================================================================================
[2025-10-14 12:02:41] Found 1 active leagues
[2025-10-14 12:02:41] 
[2025-10-14 12:02:41] ================================================================================
[2025-10-14 12:02:41] PROCESSING LEAGUE: 10 - ליגת Winner סל
[2025-10-14 12:02:41] ================================================================================
[2025-10-14 12:02:41] ✅ Loaded global team mapping: 93 teams, 316 name variations
[2025-10-14 12:02:41]    League 1: 16 teams, 49 variations
[2025-10-14 12:02:41] [WINNER] Initialized WinnerScraper with 5 boards
[2025-10-14 12:02:41] [WINNER] ============================================================
[

SystemExit: 1

C:\Users\navon\AppData\Local\Programs\Python\Python312\Lib\site-packages\IPython\core\interactiveshell.py:3558: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
# -*- coding: utf-8 -*-
# Print a filtered tree: folders + CSV files only, plus a flat CSV list with sizes.

from pathlib import Path

# ---------- Settings ----------
ROOT = Path(".").resolve()
MAX_DEPTH = 5          # עומק מקסימלי לסריקה
SHOW_HIDDEN = False    # להציג תיקיות/קבצים שמתחילים בנקודה?

# ---------- Helpers ----------
def is_hidden(p: Path) -> bool:
    return any(part.startswith(".") for part in p.parts)

def within_depth(root: Path, p: Path, max_depth: int) -> bool:
    return len(p.relative_to(root).parts) <= max_depth

def dir_children(root: Path, d: Path):
    """Return (dirs, csvs) immediate children (not recursive), filtered."""
    dirs, csvs = [], []
    for child in sorted(d.iterdir()):
        if not SHOW_HIDDEN and is_hidden(child):
            continue
        if child.is_dir():
            dirs.append(child)
        elif child.is_file() and child.suffix.lower() == ".csv":
            csvs.append(child)
    return dirs, csvs

def contains_relevant_descendants(root: Path, d: Path, max_depth: int) -> bool:
    """Does this directory (recursively) contain any dir/csv within max_depth?"""
    for p in d.rglob("*"):
        # stop at max depth
        if not within_depth(root, p, max_depth):
            continue
        if not SHOW_HIDDEN and is_hidden(p):
            continue
        if p.is_dir() or (p.is_file() and p.suffix.lower() == ".csv"):
            return True
    return False

def print_tree(root: Path, max_depth: int):
    print(f"ROOT: {root}\n")
    # We print the root name then recurse
    print(f"📁 {root.name}")
    def recurse(d: Path, prefix: str, depth: int):
        if depth >= max_depth:
            return
        dirs, csvs = dir_children(root, d)

        # Filter out directories that do not contain any relevant descendants within depth
        pruned_dirs = []
        for sd in dirs:
            if within_depth(root, sd, max_depth) and (contains_relevant_descendants(root, sd, max_depth) or list(dir_children(root, sd)[1])):
                pruned_dirs.append(sd)

        # Build combined list: dirs first, then CSVs
        items = pruned_dirs + csvs
        for i, item in enumerate(items):
            is_last = (i == len(items) - 1)
            branch = "└─ " if is_last else "├─ "
            if item.is_dir():
                print(prefix + branch + "📁 " + item.name)
                new_prefix = prefix + ("   " if is_last else "│  ")
                recurse(item, new_prefix, depth + 1)
            else:
                print(prefix + branch + "📄 " + item.name)
    recurse(root, "", 0)

def human_size(num_bytes: int) -> str:
    for unit in ["B","KB","MB","GB","TB"]:
        if num_bytes < 1024.0:
            return f"{num_bytes:.0f} {unit}"
        num_bytes /= 1024.0
    return f"{num_bytes:.0f} PB"

def list_csvs(root: Path, max_depth: int):
    csvs = []
    for p in sorted(root.rglob("*.csv")):
        if not within_depth(root, p, max_depth):
            continue
        if not SHOW_HIDDEN and is_hidden(p.relative_to(root)):
            continue
        if p.is_file():
            try:
                size = human_size(p.stat().st_size)
            except Exception:
                size = "?"
            csvs.append((p, size))
    return csvs

# ---------- Run ----------
print("=== DIRECTORY TREE (folders + CSV only) ===")
print_tree(ROOT, MAX_DEPTH)

print("\n=== CSV FILES (flat list) ===")
csv_list = list_csvs(ROOT, MAX_DEPTH)
if not csv_list:
    print("(no CSV files found)")
else:
    for p, size in csv_list:
        rel = p.relative_to(ROOT)
        print(f"- {rel}  •  {size}")


In [ ]:
# -*- coding: utf-8 -*-
# מציג לכל קובץ CSV את הנתיב, כותרות ושורה אחת ראשונה בפורמט CSV
# מדלג על תיקיות מוסתרות ו-.ipynb_checkpoints, עם מפריד ברור בין קבצים

from pathlib import Path
import pandas as pd

ROOT = Path(".").resolve()
MAX_DEPTH = 5

def is_hidden_or_checkpoint(path: Path) -> bool:
    """בודק האם קובץ או תיקייה מוסתרים או בתוך .ipynb_checkpoints"""
    return any(
        part.startswith(".") or "ipynb_checkpoints" in part
        for part in path.parts
    )

def within_depth(root: Path, path: Path, max_depth: int) -> bool:
    """בודק האם הקובץ בעומק מותר"""
    return len(path.relative_to(root).parts) <= max_depth

def print_csv_previews(root: Path, max_depth: int):
    csv_files = sorted(root.rglob("*.csv"))
    for csv_path in csv_files:
        if is_hidden_or_checkpoint(csv_path.relative_to(root)):
            continue
        if not within_depth(root, csv_path, max_depth):
            continue

        rel_path = str(csv_path.relative_to(root))
        try:
            df = pd.read_csv(csv_path, encoding="utf-8-sig", nrows=1)
        except Exception:
            try:
                df = pd.read_csv(csv_path, encoding="utf-8", nrows=1)
            except Exception as e:
                print(f"\n{rel_path}")
                print(f"Error reading file: {e}")
                print("=" * 60)
                continue

        print(f"\n{rel_path}")  # כותרת עם נתיב הקובץ
        print(",".join(df.columns))  # שורת כותרות
        if not df.empty:
            print(",".join(map(str, df.iloc[0].tolist())))  # שורה אחת מהנתונים
        print("=" * 60)  # מפריד בין קבצים

    print("\n✅ הסתיים בהצלחה.")

# הפעלה
print_csv_previews(ROOT, MAX_DEPTH)


In [ ]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
"""
Test Live Mapping
=================
בדיקה עם הקוד המעודכן בדיוק
"""

import pandas as pd
import html

teams_file = "data/teams.csv"

print(f"\n{'='*80}")
print("Loading with COMPOSITE KEY: (variation, league_id)")
print(f"{'='*80}\n")

df = pd.read_csv(teams_file, encoding='utf-8-sig')
print(f"✓ Loaded {len(df)} rows\n")

# זיהוי עמודות
columns = df.columns.tolist()
column_mapping = {
    'team_id': ['Team_ID', 'team_id', 'TeamID'],
    'league_id': ['League_ID', 'league_id', 'LeagueID'],
    'club_name': ['Team_Name', 'club_name', 'team_name', 'name'],
    'variations': ['name_variations', 'variations', 'Variations'],
}

actual_columns = {}
for key, possible_names in column_mapping.items():
    for name in possible_names:
        if name in columns:
            actual_columns[key] = name
            break

print(f"Detected columns: {actual_columns}\n")

# יצירת מיפוי עם מפתח מורכב
mapping = {}
teams_count = 0

for _, row in df.iterrows():
    team_id = row[actual_columns['team_id']] if 'team_id' in actual_columns else None
    league_id = row[actual_columns['league_id']] if 'league_id' in actual_columns else None
    club_name = row[actual_columns['club_name']] if 'club_name' in actual_columns else None
    
    if pd.isna(team_id) or pd.isna(league_id) or pd.isna(club_name):
        continue
    
    try:
        league_id_int = int(league_id)
        if league_id_int == 0:
            continue
    except:
        continue
    
    teams_count += 1
    
    variations = row[actual_columns['variations']] if 'variations' in actual_columns else club_name
    
    club_name = html.unescape(str(club_name)).strip()
    variations_str = html.unescape(str(variations)).strip()
    
    variation_list = []
    for v in variations_str.split('|'):
        v_clean = html.unescape(v.strip())
        if v_clean:
            variation_list.append(v_clean)
    
    team_info = {
        'team_id': int(team_id),
        'league_id': int(league_id_int),
        'club_name': club_name,
        'short_name': club_name,
        'bg_color': '#000000',
        'text_color': '#FFFFFF',
        'all_variations': variation_list
    }
    
    # ⭐ מפתח מורכב!
    for variation in variation_list:
        if variation:
            key = (variation, int(league_id_int))
            if key not in mapping:
                mapping[key] = team_info

print(f"✓ Total teams: {teams_count}")
print(f"✓ Total mapping entries: {len(mapping)}\n")

# בדיקת ליגה 1
league_1_keys = [k for k in mapping.keys() if k[1] == 1]
print(f"✓ League 1 has {len(league_1_keys)} variations\n")

if league_1_keys:
    print("First 10 variations from League 1:")
    for i, key in enumerate(league_1_keys[:10]):
        var, lid = key
        info = mapping[key]
        print(f"  {i+1}. ('{var}', {lid}) → team_id={info['team_id']}, club_name='{info['club_name']}'")

print(f"\n{'='*80}")
print("Testing specific lookups:")
print(f"{'='*80}\n")

test_cases = [
    ('אליצור יבנה', 1),
    ('מכבי חיפה גיא נתן', 1),
    ('מכבי אשדוד', 1),
    ('הפועל חיפה', 1),
    ('מכבי רחובות', 1),
]

for team_name, league_id in test_cases:
    key = (team_name, league_id)
    if key in mapping:
        info = mapping[key]
        print(f"✓ ({team_name!r}, {league_id}) → team_id={info['team_id']}")
    else:
        print(f"✗ ({team_name!r}, {league_id}) NOT FOUND")
        
        # חיפוש באיזו ליגה זה כן נמצא
        found_in = []
        for k in mapping.keys():
            if k[0] == team_name:
                found_in.append(k[1])
        if found_in:
            print(f"  Found in leagues: {found_in}")

print(f"\n{'='*80}\n")

In [ ]:
"""
Clear Python Cache - Jupyter Notebook
======================================
מחיקת כל קבצי ה-cache של Python
"""

import os
import shutil
from pathlib import Path

def clear_python_cache(root_dir='.'):
    """
    מחיקת כל קבצי ה-cache של Python
    
    Args:
        root_dir: תיקייה להתחיל ממנה (ברירת מחדל: נוכחית)
    """
    deleted_count = 0
    
    # מחיקת תיקיות __pycache__
    print("🔍 מחפש תיקיות __pycache__...")
    for pycache_dir in Path(root_dir).rglob('__pycache__'):
        try:
            shutil.rmtree(pycache_dir)
            print(f"   ✅ נמחק: {pycache_dir}")
            deleted_count += 1
        except Exception as e:
            print(f"   ❌ שגיאה במחיקת {pycache_dir}: {e}")
    
    # מחיקת קבצי .pyc
    print("\n🔍 מחפש קבצי .pyc...")
    pyc_count = 0
    for pyc_file in Path(root_dir).rglob('*.pyc'):
        try:
            pyc_file.unlink()
            pyc_count += 1
        except Exception as e:
            print(f"   ❌ שגיאה במחיקת {pyc_file}: {e}")
    
    if pyc_count > 0:
        print(f"   ✅ נמחקו {pyc_count} קבצי .pyc")
    
    # מחיקת קבצי .pyo
    print("\n🔍 מחפש קבצי .pyo...")
    pyo_count = 0
    for pyo_file in Path(root_dir).rglob('*.pyo'):
        try:
            pyo_file.unlink()
            pyo_count += 1
        except Exception as e:
            print(f"   ❌ שגיאה במחיקת {pyo_file}: {e}")
    
    if pyo_count > 0:
        print(f"   ✅ נמחקו {pyo_count} קבצי .pyo")
    
    print("\n" + "="*60)
    print(f"✅ סיים! נמחקו {deleted_count} תיקיות __pycache__")
    print(f"✅ נמחקו {pyc_count + pyo_count} קבצים מקומפלים")
    print("="*60)


# הרצה
print("\n" + "="*60)
print("מחיקת Python Cache")
print("="*60 + "\n")

clear_python_cache()

print("\n💡 כעת הרץ את הקוד מחדש:")
print("   !python main.py --league 1")

In [ ]:
# -*- coding: utf-8 -*-
"""
Migration Script - CSV to Supabase
===================================
מעביר את כל הנתונים הקיימים מ-CSV ל-Supabase
"""

import os
import pandas as pd
from supabase_uploader import SupabaseUploader
from config import LEAGUES
from utils import log_message

class DataMigration:
    """מחלקה להעברת נתונים"""
    
    def __init__(self):
        """אתחול"""
        self.uploader = SupabaseUploader()
        self.stats = {
            'leagues': 0,
            'teams': 0,
            'players': 0,
            'history': 0,
            'games': 0,
            'quarters': 0,
            'player_stats': 0,
            'team_stats': 0,
            'player_averages': 0,
            'team_averages': 0,
            'opponent_averages': 0
        }
    
    def migrate_all(self, league_ids=None):
        """
        העברת כל הנתונים
        
        Args:
            league_ids: רשימת league_ids להעברה (None = כולם)
        """
        log_message("="*60)
        log_message("🚀 STARTING MIGRATION TO SUPABASE")
        log_message("="*60)
        
        # Test connection
        if not self.uploader.test_connection():
            log_message("❌ Cannot connect to Supabase!")
            return False
        
        # Get leagues to migrate
        if league_ids is None:
            leagues_to_migrate = {lid: cfg for lid, cfg in LEAGUES.items() if cfg.get('active', False)}
        else:
            leagues_to_migrate = {lid: cfg for lid, cfg in LEAGUES.items() if lid in league_ids}
        
        log_message(f"📊 Migrating {len(leagues_to_migrate)} leagues")
        log_message("")
        
        # Migrate global data first
        self._migrate_global_leagues()
        self._migrate_global_teams()
        self._migrate_global_players()
        
        # Migrate each league
        for league_id, config in leagues_to_migrate.items():
            log_message("="*60)
            log_message(f"[{config['code'].upper()}] Migrating: {config['name']}")
            log_message("="*60)
            
            self._migrate_league_data(league_id, config)
        
        # Print stats
        self._print_stats()
        
        log_message("="*60)
        log_message("✅ MIGRATION COMPLETED!")
        log_message("="*60)
        
        return True
    
    def _migrate_global_leagues(self):
        """העברת טבלת leagues"""
        log_message("📋 Migrating leagues...")
        
        leagues_file = 'data/leagues.csv'
        if not os.path.exists(leagues_file):
            log_message("⚠️  leagues.csv not found, creating from config")
            # Create from config
            for league_id, config in LEAGUES.items():
                if config.get('active', False):
                    league_data = {
                        'league_id': int(league_id),
                        'name': config['name'],
                        'name_en': config.get('name_en', ''),
                        'country': config.get('country', 'Israel'),
                        'season': config.get('season', ''),
                        'url': config['url'],
                        'is_active': True
                    }
                    self.uploader.upsert_league(league_data)
                    self.stats['leagues'] += 1
        else:
            df = pd.read_csv(leagues_file, encoding='utf-8-sig')
            for _, row in df.iterrows():
                league_data = {
                    'league_id': int(row['league_id']),
                    'name': row['name'],
                    'name_en': row.get('name_en', ''),
                    'country': row.get('country', 'Israel'),
                    'season': row.get('season', ''),
                    'url': row.get('url', ''),
                    'is_active': bool(row.get('is_active', False))
                }
                self.uploader.upsert_league(league_data)
                self.stats['leagues'] += 1
        
        log_message(f"✅ Migrated {self.stats['leagues']} leagues")
    
    def _migrate_global_teams(self):
        """העברת טבלת teams"""
        log_message("🏀 Migrating teams...")
        
        teams_file = 'data/teams.csv'
        if not os.path.exists(teams_file):
            log_message("⚠️  teams.csv not found, skipping")
            return
        
        df = pd.read_csv(teams_file, encoding='utf-8-sig')
        teams_data = []
        
        for _, row in df.iterrows():
            # Skip invalid teams
            if pd.isna(row['Team_ID']) or pd.isna(row['League_ID']):
                continue
            if int(row['League_ID']) == 0:
                continue
            
            teams_data.append({
                'team_id': int(row['Team_ID']),
                'league_id': int(row['League_ID']),
                'team_name': row['Team_Name'],
                'short_name': row.get('short_name', row['Team_Name']),
                'bg_color': row.get('bg_color', '#000000'),
                'text_color': row.get('text_color', '#FFFFFF'),
                'name_variations': row.get('name_variations', '')
            })
        
        if teams_data:
            self.uploader.upsert_teams(teams_data)
            self.stats['teams'] = len(teams_data)
        
        log_message(f"✅ Migrated {self.stats['teams']} teams")
    
    def _migrate_global_players(self):
        """העברת טבלת players"""
        log_message("👤 Migrating players...")
        
        players_file = 'data/players.csv'
        if not os.path.exists(players_file):
            log_message("⚠️  players.csv not found, skipping")
            return
        
        df = pd.read_csv(players_file, encoding='utf-8-sig')
        players_data = []
        
        for _, row in df.iterrows():
            if pd.isna(row['player_id']):
                continue
            
            players_data.append({
                'player_id': str(row['player_id']),
                'name': row['name'],
                'current_team_id': int(row['current_team_id']) if pd.notna(row['current_team_id']) else None,
                'league_id': int(row['league_id']),
                'date_of_birth': row['date_of_birth'] if pd.notna(row['date_of_birth']) else None,
                'height': float(row['height']) if pd.notna(row['height']) else None,
                'jersey_number': int(row['jersey_number']) if pd.notna(row['jersey_number']) else None
            })
        
        if players_data:
            self.uploader.upsert_players(players_data)
            self.stats['players'] = len(players_data)
        
        log_message(f"✅ Migrated {self.stats['players']} players")
    
    def _migrate_league_data(self, league_id, config):
        """העברת נתוני ליגה ספציפית"""
        league_code = config['code']
        data_folder = config['data_folder']
        games_folder = config['games_folder']
        
        # Player details (if not in global file)
        self._migrate_player_details(league_id, league_code, data_folder)
        
        # Player history
        self._migrate_player_history(league_id, league_code, data_folder)
        
        # Games
        self._migrate_games(league_id, games_folder)
        
        # Game quarters
        self._migrate_game_quarters(league_id, games_folder)
        
        # Game player stats
        self._migrate_game_player_stats(league_id, games_folder)
        
        # Game team stats
        self._migrate_game_team_stats(league_id, games_folder)
        
        # Averages
        self._migrate_player_averages(league_id, league_code, data_folder)
        self._migrate_team_averages(league_id, league_code, data_folder)
        self._migrate_opponent_averages(league_id, league_code, data_folder)
    
    def _migrate_player_details(self, league_id, league_code, data_folder):
        """העברת פרטי שחקנים"""
        file_path = os.path.join(data_folder, f"{league_code}_player_details.csv")
        if not os.path.exists(file_path):
            return
        
        log_message(f"  📝 Player details...")
        df = pd.read_csv(file_path, encoding='utf-8-sig')
        
        players_data = []
        for _, row in df.iterrows():
            players_data.append({
                'player_id': str(row['player_id']),
                'name': row['Name'],
                'current_team_id': int(row['team_id']) if pd.notna(row['team_id']) else None,
                'league_id': int(league_id),
                'date_of_birth': row['Date Of Birth'] if pd.notna(row['Date Of Birth']) else None,
                'height': float(row['Height']) if pd.notna(row['Height']) else None,
                'jersey_number': int(row['Number']) if pd.notna(row['Number']) else None
            })
        
        if players_data:
            self.uploader.upsert_players(players_data)
            log_message(f"  ✅ {len(players_data)} players")
    
    def _migrate_player_history(self, league_id, league_code, data_folder):
        """העברת היסטוריית שחקנים"""
        file_path = os.path.join(data_folder, f"{league_code}_player_history.csv")
        if not os.path.exists(file_path):
            return
        
        log_message(f"  📚 Player history...")
        df = pd.read_csv(file_path, encoding='utf-8-sig')
        
        history_data = []
        for _, row in df.iterrows():
            player_id = str(row['player_id'])
            
            # Extract seasons from columns
            for col in df.columns:
                if col not in ['player_id', 'Name', 'Current Team', 'team_id', 'league_id',
                               'Date Of Birth', 'Height', 'Number']:
                    season = col
                    if pd.notna(row[col]) and str(row[col]).strip():
                        team_league = str(row[col])
                        team_name = team_league.split('(')[0].strip() if '(' in team_league else team_league
                        league_name = team_league.split('(')[1].replace(')', '').strip() if '(' in team_league else ''
                        
                        history_data.append({
                            'player_id': player_id,
                            'season': season,
                            'team_name': team_name,
                            'league_name': league_name,
                            'league_id': int(league_id)
                        })
        
        if history_data:
            self.uploader.upsert_player_history(history_data)
            self.stats['history'] += len(history_data)
            log_message(f"  ✅ {len(history_data)} history records")
    
    def _migrate_games(self, league_id, games_folder):
        """העברת משחקים"""
        file_path = os.path.join(games_folder, 'games_schedule.csv')
        if not os.path.exists(file_path):
            return
        
        log_message(f"  🏆 Games...")
        df = pd.read_csv(file_path, encoding='utf-8-sig')
        
        games_data = []
        for _, row in df.iterrows():
            games_data.append({
                'game_id': str(row['gameid']),
                'league_id': int(league_id),
                'code': str(row['Code']) if pd.notna(row.get('Code')) else None,
                'week_day': row.get('Week Day', ''),
                'date': row['Date'] if pd.notna(row['Date']) else None,
                'round': str(row['Round']) if pd.notna(row.get('Round')) else None,
                'time': row.get('Time', ''),
                'home_team': row['Home Team'],
                'home_team_code': row.get('Home Team Code', ''),
                'home_team_id': int(row['home_team_id']) if pd.notna(row.get('home_team_id')) else None,
                'away_team': row['Away Team'],
                'away_team_code': row.get('Away Team Code', ''),
                'away_team_id': int(row['away_team_id']) if pd.notna(row.get('away_team_id')) else None,
                'venue': row.get('Venue', ''),
                'home_score': int(row['Home Score']) if pd.notna(row.get('Home Score')) and str(row['Home Score']).replace('.','').isdigit() else None,
                'away_score': int(row['Away Score']) if pd.notna(row.get('Away Score')) and str(row['Away Score']).replace('.','').isdigit() else None,
                'arena': row.get('Arena', ''),
                'status': 'completed' if pd.notna(row.get('Home Score')) and str(row.get('Home Score')).strip() != '' else 'scheduled'
            })
        
        if games_data:
            self.uploader.upsert_games(games_data)
            self.stats['games'] += len(games_data)
            log_message(f"  ✅ {len(games_data)} games")
    
    def _migrate_game_quarters(self, league_id, games_folder):
        """העברת רבעי משחק"""
        file_path = os.path.join(games_folder, 'game_quarters.csv')
        if not os.path.exists(file_path):
            return
        
        log_message(f"  🔢 Quarters...")
        df = pd.read_csv(file_path, encoding='utf-8-sig')
        
        quarters_data = df.to_dict('records')
        
        if quarters_data:
            self.uploader.upsert_game_quarters(quarters_data)
            self.stats['quarters'] += len(quarters_data)
            log_message(f"  ✅ {len(quarters_data)} quarters")
    
    def _migrate_game_player_stats(self, league_id, games_folder):
        """העברת סטטיסטיקות שחקן במשחק"""
        file_path = os.path.join(games_folder, 'game_player_stats.csv')
        if not os.path.exists(file_path):
            return
        
        log_message(f"  📊 Player stats...")
        df = pd.read_csv(file_path, encoding='utf-8-sig')
        
        stats_data = df.to_dict('records')
        
        if stats_data:
            self.uploader.upsert_game_player_stats(stats_data)
            self.stats['player_stats'] += len(stats_data)
            log_message(f"  ✅ {len(stats_data)} player stats")
    
    def _migrate_game_team_stats(self, league_id, games_folder):
        """העברת סטטיסטיקות קבוצה במשחק"""
        file_path = os.path.join(games_folder, 'game_team_stats.csv')
        if not os.path.exists(file_path):
            return
        
        log_message(f"  📈 Team stats...")
        df = pd.read_csv(file_path, encoding='utf-8-sig')
        
        stats_data = df.to_dict('records')
        
        if stats_data:
            self.uploader.upsert_game_team_stats(stats_data)
            self.stats['team_stats'] += len(stats_data)
            log_message(f"  ✅ {len(stats_data)} team stats")
    
    def _migrate_player_averages(self, league_id, league_code, data_folder):
        """העברת ממוצעי שחקנים"""
        file_path = os.path.join(data_folder, f"{league_code}_player_averages.csv")
        if not os.path.exists(file_path):
            return
        
        log_message(f"  📉 Player averages...")
        df = pd.read_csv(file_path, encoding='utf-8-sig')
        
        avg_data = df.to_dict('records')
        
        if avg_data:
            self.uploader.upsert_player_averages(avg_data)
            self.stats['player_averages'] += len(avg_data)
            log_message(f"  ✅ {len(avg_data)} player averages")
    
    def _migrate_team_averages(self, league_id, league_code, data_folder):
        """העברת ממוצעי קבוצות"""
        file_path = os.path.join(data_folder, f"{league_code}_team_averages.csv")
        if not os.path.exists(file_path):
            return
        
        log_message(f"  📊 Team averages...")
        df = pd.read_csv(file_path, encoding='utf-8-sig')
        
        avg_data = df.to_dict('records')
        
        if avg_data:
            self.uploader.upsert_team_averages(avg_data)
            self.stats['team_averages'] += len(avg_data)
            log_message(f"  ✅ {len(avg_data)} team averages")
    
    def _migrate_opponent_averages(self, league_id, league_code, data_folder):
        """העברת ממוצעי יריבים"""
        file_path = os.path.join(data_folder, f"{league_code}_opponent_averages.csv")
        if not os.path.exists(file_path):
            return
        
        log_message(f"  🛡️ Opponent averages...")
        df = pd.read_csv(file_path, encoding='utf-8-sig')
        
        avg_data = df.to_dict('records')
        
        if avg_data:
            self.uploader.upsert_opponent_averages(avg_data)
            self.stats['opponent_averages'] += len(avg_data)
            log_message(f"  ✅ {len(avg_data)} opponent averages")
    
    def _print_stats(self):
        """הדפסת סטטיסטיקות"""
        log_message("")
        log_message("="*60)
        log_message("📊 MIGRATION STATISTICS")
        log_message("="*60)
        
        for key, value in self.stats.items():
            if value > 0:
                log_message(f"  {key.replace('_', ' ').title()}: {value:,}")
        
        total = sum(self.stats.values())
        log_message("")
        log_message(f"  TOTAL RECORDS: {total:,}")
        log_message("="*60)


# ============================================
# MAIN EXECUTION
# ============================================

if __name__ == "__main__":
    import argparse
    
    parser = argparse.ArgumentParser(description='Migrate CSV data to Supabase')
    parser.add_argument('--league', type=str, help='Specific league ID to migrate')
    parser.add_argument('--all', action='store_true', help='Migrate all active leagues')
    parser.add_argument('--test', action='store_true', help='Test connection only')
    
    args = parser.parse_args()
    
    migrator = DataMigration()
    
    if args.test:
        # Test connection
        log_message("🧪 Testing Supabase connection...")
        if migrator.uploader.test_connection():
            log_message("✅ Connection successful!")
        else:
            log_message("❌ Connection failed!")
    
    elif args.all or args.league:
        # Migrate data
        league_ids = [args.league] if args.league else None
        migrator.migrate_all(league_ids)
    
    else:
        # Show help
        parser.print_help()
        print("\nExamples:")
        print("  python migrate_to_supabase.py --test              # Test connection")
        print("  python migrate_to_supabase.py --all               # Migrate all leagues")
        print("  python migrate_to_supabase.py --league 1          # Migrate league 1 only")

In [ ]:
# Cell 1: Import ספציפי
from supabase import create_client, Client

# Cell 2: טען credentials
import os
from dotenv import load_dotenv

load_dotenv()

url = os.getenv('SUPABASE_URL')
key = os.getenv('SUPABASE_KEY')

# Cell 3: בדוק שהם קיימים
if not url or not key:
    print("❌ חסר URL או KEY!")
else:
    print(f"✅ URL: {url}")
    print(f"✅ Key מתחיל ב: {key[:20]}...")

# Cell 4: חיבור עם try-except
try:
    supabase: Client = create_client(url, key)
    print("✅ Connected!")
    
    # בדיקה
    result = supabase.table('leagues').select("count", count="exact").execute()
    print(f"✅ יש {result.count} ליגות")
    
except Exception as e:
    print(f"❌ Error: {e}")